# Trees

## Introduction to optimization and operations research

Michel Bierlaire


In [ ]:

import random
from itertools import combinations
from typing import Any

import numpy as np
from networkx import (
    Graph,
    random_tree,
    spring_layout,
    draw,
    find_cycle,
    NetworkXNoCycle,
    is_connected,
    connected_components,
)
from matplotlib import pyplot as plt


In this lab, you will explore **trees** and their properties using NetworkX.
You will generate random trees, visualize them, and practice checking connectivity
and the absence of cycles. You will implement small utilities to detect cycles,
test connectedness, and add/remove edges, then observe how these operations affect
the graph (for example, adding one edge to a tree creates exactly one cycle, while
removing one edge disconnects it). The goal is to build intuition about how local
edge changes alter global structure, preparing you for later topics such as flows,
cuts, and shortest paths on networks, and for formulating network models as a
**linear optimization problem** when we introduce capacities and costs.

A node in the_network can be of any type. In this script, we use int.

In [ ]:
Node = Any
Edge = tuple[Node, Node]



Here is a function to plot a graph.

In [ ]:
def visualize_graph(graph: Graph, positions: dict[Node, np.array]) -> None:
    """Plot a graph

    :param graph: graph to be plotted.
    :param positions: coordinates of the nodes.
    """
    draw(
        graph,
        positions,
        with_labels=True,
        node_color='lightblue',
        node_size=500,
        font_size=10,
        edge_color='gray',
    )
    plt.title("Random Tree")
    plt.show()



Example: Create a random tree with 10 nodes

In [ ]:
n = 10
tree: Graph = random_tree(n)
pos = spring_layout(tree)



Visualize the graph

In [ ]:
visualize_graph(tree, pos)



Implement a function that detects a cycle in a graph

In [ ]:
def detect_cycle(graph: Graph) -> list[Edge] | None:
    """Detect if a graph contains a cycle

    :param graph: graph to analyze
    :param graph: graph to analyze
    :return: the list of arcs in the cycle, or None if None has been found.
    """
    try:
        # This will return a list of cycles
        a_cycle = find_cycle(graph, orientation="ignore")
        return a_cycle
    except NetworkXNoCycle:
        return None



Check if the tree contains a cycle.
By definition, a tree never contains any cycle.

In [ ]:
cycle = detect_cycle(tree)
if cycle:
    print(f'A cycle was detected: {cycle}')
else:
    print('No cycle was detected.')



Implement a function that checks if the graph is connected.
If not, identify two connected components, and return one node from each.

In [ ]:
def is_graph_connected(graph: Graph) -> tuple[Node, Node] | None:
    """
    Determines if the given graph is connected.

    :param graph: an undirected graph (Graph)
    :return: two disconnected nodes, or None if the graph is disconnected.
    """
    if is_connected(graph):
        return None

    # Find the connected components
    components = list(connected_components(graph))

    # Get two nodes from different components
    # Choose the first node from the first component and the first node from the second component
    node1 = next(iter(components[0]))
    node2 = next(iter(components[1]))
    return node1, node2



Check if the tree is connected.
By definition, a tree is always connected.

In [ ]:
result = is_graph_connected(tree)
message = (
    'The graph is connected'
    if result is None
    else f'Nodes {result[0]} and {result[1]} are not connected'
)
print(message)



Now, write a function that adds a random arc to a graph

In [ ]:
def add_edge(graph: Graph) -> Edge:
    """Add a random arc in a graph

    :param graph: graph to complete.
    :return: added arc.
    """
    # First, check the missing arcs in the graph.
    nodes = list(graph.nodes)
    missing_edges = list()
    for upstream_node, downstream_node in combinations(nodes, 2):
        if not graph.has_edge(upstream_node, downstream_node):
            missing_edges.append((upstream_node, downstream_node))
    if not missing_edges:
        print(f'The graph is complete. No arc can be added.')

    # Choose randomly a missing arc and add it to the graph
    up_stream_node, downstream_node = (
        random.choice(missing_edges)
    )
    # Add the selected arc to the graph.
    graph.add_edge(up_stream_node, downstream_node)
    return up_stream_node, downstream_node



Let's add an arc to the tree

In [ ]:
u, v = add_edge(tree)
print(f'An arc between nodes {u} and {v} has been added.')


We plot the modified tree

In [ ]:
visualize_graph(tree, pos)


Check the presence of a cycle, and the connectivity of the modified tree.
Cycle:
In a tree, adding a single arc always creates a cycle.

In [ ]:
cycle = detect_cycle(tree)
if cycle:
    print(f'A cycle was detected: {cycle}')
else:
    print('No cycle was detected.')


Connectivity:
As the graph was connected, adding an arc cannot disconnect the graph.

In [ ]:
result = is_graph_connected(tree)
message = (
    'The graph is connected'
    if result is None
    else f'Nodes {result[0]} and {result[1]} are not connected'
)
print(message)



Now, write a function that removes a random arc to a graph

In [ ]:
def remove_edge(graph: Graph) -> Edge | None:
    """
    Removes a random arc from the given graph.

    :param graph: A NetworkX graph
    """
    if graph.number_of_edges() == 0:
        print('The graph has no arc to remove.')
        return None

    # Select a random arc
    random_edge: Edge = random.choice(
        list(graph.edges)
    )
    # Remove the selected arc. Note that the `*random_edge` unpacks the tuple into its two components.
    graph.remove_edge(*random_edge)
    print(f'Edge {random_edge} removed.')
    return random_edge



We generate another random tree

In [ ]:
tree = random_tree(n)
pos = spring_layout(tree)
visualize_graph(tree, pos)


Check the presence of a cycle, and the connectivity of the modified tree.

Cycle:
By definition of a tree, it does not contain any cycle.

In [ ]:
cycle = detect_cycle(tree)
if cycle:
    print(f'A cycle was detected: {cycle}')
else:
    print('No cycle was detected.')


Connectivity:
By definition of a tree, it is connected.

In [ ]:
result = is_graph_connected(tree)
message = (
    'The graph is connected'
    if result is None
    else f'Nodes {result[0]} and {result[1]} are not connected'
)
print(message)


We now remove a random arc.

In [ ]:
u, v = remove_edge(tree)
print(f'Edge between ({u}, {v}) has been removed.')
visualize_graph(tree, pos)


Check the presence of a cycle, and the connectivity of the modified tree.

Cycle:
Removing an arc cannot create a cycle if there were none before.

In [ ]:
cycle = detect_cycle(tree)
if cycle:
    print(f'A cycle was detected: {cycle}')
else:
    print('No cycle was detected.')


Connectivity:
In a tree, removing a single arc disconnects the tree.

In [ ]:
result = is_graph_connected(tree)
message = (
    'The graph is connected'
    if result is None
    else f'Nodes {result[0]} and {result[1]} are not connected'
)
print(message)